pip install cdsapi  
pip install pygrib

In [ ]:
import os, sys, cdsapi, pygrib, calendar
from QueryHandler import QueryHandler
from shapely.geometry import Point
from dotenv import load_dotenv
from typing import Union
import sqlalchemy as sq 
import geopandas as gpd
import numpy as np

sys.path.append('../')
from DataService import DataService


load_dotenv()
PG_DB = os.getenv('POSTGRES_DB')
PG_ADDR = os.getenv('POSTGRES_ADDR')
PG_PORT = os.getenv('POSTGRES_PORT')
PG_USER = os.getenv('POSTGRES_USER')
PG_PW = os.getenv('POSTGRES_PW')

MIN_MONTH = 3
MAX_MONTH = 12

MIN_YEAR = 1995
MAX_YEAR = 2023

In [ ]:
c = cdsapi.Client()
queryHandler = QueryHandler()
db = DataService(PG_DB, PG_ADDR, PG_PORT, PG_USER, PG_PW)

In [ ]:
# check if the copernicus table exists, if it doesnt create it
def createTable(db: DataService):
    query = sq.text(queryHandler.tableExistsReq('copernicus_satelite_data'))
    tableExists = queryHandler.readTableExists(db.execute(query))
    
    if not tableExists:
        query = sq.text(queryHandler.createCopernicusTableReq())
        db.execute(query)

In [ ]:
# check if the current point (lon, lat) is in any of the regions pulled from the database
def calcAgRegion(agRegions: gpd.GeoDataFrame, point: Point) -> str:
    area = ''   # by default it is assumed we will not find this point

    for index, region in agRegions.iterrows():
        if region['geometry'].contains(point)[0]:   # for each region, check if the point is within that areas geometry
            area = region['car_name']               # found it, update the area with the new name
            break
    
    return area

In [ ]:
def storeData(db: DataService, lon: float, lat: float, year: str, month: str, day: str, hour: str, region: str, attr: str, value: Union[float, str]):
    datetime = np.datetime64(f'{year}-{month}-{day}T{hour}')                    # creates the datetime as per np.datetime64
    query = sq.text(queryHandler.createRowExistsInDBReq(lon, lat, datetime))    # check if the current row exists in the database
    rowExists = queryHandler.readRowExistsInDB(db.execute(query))
    hour = hour.split(':')[0]                                                   # get the numeric value for the current hour

    if rowExists:
        query = sq.text(queryHandler.createUpdateRowReq(lon, lat, datetime, attr, value))   # if row exists update it
        db.execute(query)
    else:
        query = sq.text(queryHandler.createInsertRowReq(lon, lat, datetime, year, month, day, hour, region, attr, value))   # else create a new row
        db.execute(query)

In [ ]:
def processFile(grbs, db: DataService, agRegions: gpd.GeoDataFrame, year: str, month: str, day: str, hour: str, attr: str):
    listOfdata, listOfLats, listOfLons = grbs[1].data() # grbs has one dimention (for the searched attr) but the variables it returns are 2D arrays
                                                        # listOfData = data, listOfLats = lats, listOfLons = lons
    for listIndex, list in enumerate(listOfdata):   # for each list
        for dataIndex, data in enumerate(list):     # for each data point in the current list
            x = listOfLons[listIndex][dataIndex]    # extract the x value
            y = listOfLats[listIndex][dataIndex]    # extract the y value 

            point = Point(x, y)                             # create geometry for the current point
            point = gpd.GeoSeries(point, crs='EPSG:4326')   # turn it into a geoseries and sets the projection to common lon, lat
            point = point.to_crs(crs='EPSG:3347')           # changes the projection to match the agriculture regions
            region = calcAgRegion(agRegions, point)         # check if the point belongs to an area of interest, if so return it

            if region:
                if(len(month) != 2):        # ensures the month is formatted accordingly to satisfy np.datetime64
                    month = '0' + month

                if(len(day) != 2):          # ensures the day is formatted accordingly to satisfy np.datetime64
                    day = '0' + day

                if(np.ma.is_masked(data)):  # if the data is masked/null, set it to null for database storage purposes
                    data = 'NULL'

                storeData(db, x, y, year, month, day, hour, region, attr, data)

In [ ]:
# loads the agriculture regions from the datbase (projection is EPSG:3347)
def loadGeometry(conn: sq.engine.Connection) -> gpd.GeoDataFrame:
    query = sq.text('select car_name, geometry FROM public.census_ag_regions')
    agRegions = gpd.GeoDataFrame.from_postgis(query, conn, crs='EPSG:3347', geom_col='geometry')

    return agRegions
    

In [ ]:
years = [str(year) for year in range(MIN_YEAR, MAX_YEAR + 1)]       # the year range we want to pull data from
months = [str(month) for month in range(MIN_MONTH, MAX_MONTH + 1)]  # the month range we want to pull data from

attrs = [                                                           # the attributes we want to pull data for
    '2m_dewpoint_temperature', '2m_temperature', 'evaporation_from_bare_soil', 'skin_reservoir_content', 'skin_temperature',
    'snowmelt', 'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3', 'soil_temperature_level_4',
    'surface_net_solar_radiation', 'surface_pressure', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 
    'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4'
]

hours = [                                                           # the hours we want to pull data from
    '00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00','12:00', '13:00', 
    '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'
]


conn = db.connect()             # connect to the database

createTable(db)                 # check the tables, if necessary make a new table for the data
agRegions = loadGeometry(conn)  # load the geometry from the database

for year in years:
    print(f'Pulling data for {year} ...')

    for month in months:
        numDays = calendar.monthrange(int(year), int(month))[1]
        days = [str(day) for day in range(1, numDays + 1)]

        for day in days:
            for hour in hours:
                for attr in attrs:
                    c.retrieve(                     # downloads files with modularized data: individual year, month, day, hour, attribute
                        'reanalysis-era5-land',
                        {
                            'format': 'grib',
                            'variable': [attr],
                            'year': year,
                            'month': month,
                            'day': [day],
                            'time': [hour],
                            'area': [61, -125, 48, -88],
                        },
                        'download.grib'
                    )

                    if attr == '2m_dewpoint_temperature' or attr == '2m_temperature':   # since columns cannot start with numeric values, remove them
                        attr = attr[3:]

                    # read the file, process it, close it, delete it then go onto the next set of data
                    grbs = pygrib.open('download.grib')
                    processFile(grbs, db, agRegions, year, month, day, hour, attr)
                    grbs.close()
                    os.remove('download.grib')

print('[SUCCESS] data pulled successfully')
db.cleanup()